This code no longer works very well- they added some bot detection and will have a captcha show up after a few runs. 

In [ ]:
from lxml import html
import requests
import pandas
import bs4
import urllib
import html5lib
from contextlib import closing
from urllib2 import urlopen
from splinter import Browser

In [ ]:
br = Browser('chrome')
br.visit('http://www.zulily.com')

In [ ]:
br.find_by_css('#loginModal > div.modal-dialog.login.open > div > div.modal-header > div.register-footer.step1 > a').click()
username2='login here'
password='password here'
br.find_by_id('signin_email').first.fill(username2)
br.find_by_id('signin_password').first.fill(password)
br.find_by_css('#loginForm > div.clearfix > button').click()

In [ ]:
br.find_by_id('signin_email').first.fill(username2)
br.find_by_id('signin_password').first.fill(password)
br.find_by_css('#loginForm > div.clearfix > button').click()

In [ ]:
#make list of category urls
cat_list = ['https://www.zulily.com/girls/?ref=header']
cat_list.append('https://www.zulily.com/boys/?ref=header')
cat_list.append('https://www.zulily.com/women/?ref=header')
cat_list.append('https://www.zulily.com/womens-plus/?ref=header')
cat_list.append('https://www.zulily.com/men/?ref=header')
cat_list.append('https://www.zulily.com/shoes/?ref=header')
cat_list.append('https://www.zulily.com/health-and-beauty/?ref=header')
cat_list.append('https://www.zulily.com/baby-maternity/?ref=header')
cat_list.append('https://www.zulily.com/toys-playtime/?ref=header')
cat_list.append('https://www.zulily.com/home/?ref=header')

In [ ]:
events_list = []

for i in range(0,len(cat_list)):
    br.visit(cat_list[i])
    tree = html.fromstring(br.html)
    temp_list = tree.xpath('//*[@id="home-page-content"]/div/div/*/div/a/@href')
    events_list = events_list + temp_list

events_set=set(events_list)
events_list = list(events_set)
print len(events_list)

In [ ]:
products_list = []
for i in range(0, len(events_list)):
    br.visit(events_list[i])
    tree = html.fromstring(br.html)
    temp_list = tree.xpath('//*[@id="products-list"]/*/a/@href')
    products_list = products_list + temp_list

In [ ]:
product_set = set(products_list)
product_list = list(product_set)

In [ ]:
db = MySQLdb.connect("localhost","root","password","zulily_crawls")
cursor = db.cursor()

In [ ]:
product_names = []
brands = []
prices = []
end_dates = []
ship_times=[]
for i in range(10,len(product_list)):
    br.visit(product_list[i])
    tree = html.fromstring(br.html)
    temp_name = tree.xpath('//*[@id="product-info"]/h1[1]/text()')
    temp_brand = tree.xpath('//*[@id="brand-name"]/span/text()')
    temp_price = tree.xpath('//*[@id="product-price"]/div/span/text()')
    end_date = tree.xpath('//*[@id="end-date"]/text()')
    ship_time = tree.xpath('//*[@id="product-bullets"]/li[2]/text()')
    if len(temp_name) > 0:
        temp_name = temp_name[1].strip()
    else:
        temp_name = 'empty'
    if len(temp_brand) > 0:
        temp_brand = temp_brand[0].strip()
    else:
        temp_brand = 'empty'
    
    if len(temp_price) > 0:
        temp_price = temp_price[0].strip()
    else: 
        temp_price = 'empty'
    
    if len(end_date) > 0:
        end_date = end_date[0].strip()
    else:
        end_date= 'empty'
    
    if len(ship_time) > 0:
        ship_time = ship_time[1].strip()
    else:
        ship_time = 'empty'
        
    sql = """INSERT INTO ITEM_LIST(ID, 
        URL, PRODUCT, BRAND, PRICE, ENDDATE, SHIPTIME) 
        VALUES (%d, '%s', '%s', '%s', '%s', '%s', '%s' )""" % (i , product_list[i], temp_name, temp_brand, temp_price, end_date, ship_time)
    try:
        cursor.execute(sql)
        db.commit()
    except:
        db.rollback()